# 14-day cumulative number of COVID-19 cases per 100 000 for districts (Landkreise) in Germany

At the end of the page, we provide a detailed description of how the numbers are calculated.

### Compute data

In [1]:
import datetime as dt
import pandas as pd
pd.set_option("max_rows", None)
from oscovida import get_incidence_rates_germany

## Table for all districts (=Landkreise)

In [2]:
cases_incidence, deaths_incidence = get_incidence_rates_germany()

In [3]:
cases_incidence

,14-day-sum,population,14-day-incidence-rate
Landkreis,,,
LK Ahrweiler,13,129727,10.021044
LK Aichach-Friedberg,3,133596,2.245576
LK Alb-Donau-Kreis,13,196047,6.631063
LK Altenkirchen,8,128705,6.215765
LK Altmarkkreis Salzwedel,1,83765,1.193816
LK Altötting,24,111210,21.580793
LK Alzey-Worms,8,129244,6.189842
LK Amberg-Sulzbach,2,103109,1.939695
LK Anhalt-Bitterfeld,1,159854,0.625571


## Table sorted by 14-day-incidence

In [4]:
cases_incidence.sort_values(by="14-day-incidence-rate", ascending=False)

,14-day-sum,population,14-day-incidence-rate
Landkreis,,,
SK Herne,51,156374,32.614117
LK Dingolfing-Landau,28,96217,29.100887
SK Offenbach,36,128744,27.962468
SK Bochum,93,364628,25.505447
SK Duisburg,126,498590,25.271265
SK Wuppertal,86,354382,24.267598
SK Berlin Mitte,83,384172,21.604906
LK Altötting,24,111210,21.580793
LK Olpe,29,134775,21.517344


## Tutorial: Detailed calculation for one district

In [5]:
from oscovida import fetch_data_germany, germany_get_population
import datetime

In [6]:
period = 14 # Set the period we compute the incidence rate over

In [7]:
germany = fetch_data_germany() # Downloads population data for Germany
germany = germany.rename(
    columns={"AnzahlFall": "cases", "AnzahlTodesfall": "deaths"}
) # Rename the columns to English
germany.tail() # Print example of the table format and data

,FID,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,cases,deaths,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
date,,,,,,,,,,,,,,,,,,
2020-06-09,29068504,16,Thüringen,LK Altenburger Land,A80+,M,1,0,2020/06/09 00:00:00,16077,"18.08.2020, 00:00 Uhr",0,-9,2020/05/19 00:00:00,0,1,1,Nicht übermittelt
2020-05-06,29068505,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2020/05/06 00:00:00,16077,"18.08.2020, 00:00 Uhr",0,-9,2020/05/04 00:00:00,0,1,1,Nicht übermittelt
2020-05-11,29068506,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2020/05/11 00:00:00,16077,"18.08.2020, 00:00 Uhr",0,-9,2020/05/01 00:00:00,0,1,1,Nicht übermittelt
2020-05-28,29068507,16,Thüringen,LK Altenburger Land,A80+,W,2,0,2020/05/28 00:00:00,16077,"18.08.2020, 00:00 Uhr",0,-9,2020/05/28 00:00:00,0,2,0,Nicht übermittelt
2020-06-09,29068508,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2020/06/09 00:00:00,16077,"18.08.2020, 00:00 Uhr",0,-9,2020/06/09 00:00:00,0,1,0,Nicht übermittelt


In [8]:
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
fortnight_ago = yesterday - datetime.timedelta(days=period)
periods = (fortnight_ago < germany.index) & (germany.index < yesterday) # Create a period mask
germany = germany.iloc[periods]

In [9]:
cases_germany = germany[["Landkreis", "cases"]]
cases_germany.tail() # The data for germany has the date as the index, and the location data on each column

,Landkreis,cases
date,,
2020-08-12,LK Greiz,1
2020-08-12,LK Greiz,1
2020-08-14,LK Greiz,1
2020-08-17,LK Greiz,1
2020-08-12,LK Greiz,1


In [10]:
cases_sum = (
    cases_germany.groupby("Landkreis")
    .sum()
    .rename(columns={"cases": f"{period}-day-sum"})
) # Here we sum the daily data to get total data
cases_sum.tail()

,14-day-sum
Landkreis,
SK Worms,6
SK Wuppertal,86
SK Würzburg,3
SK Zweibrücken,1
StadtRegion Aachen,40


In [11]:
population = (
    germany_get_population()
    .population
    .to_frame()
) # This gives us population per county, with county as the index
population.tail()

,population
county,
SK Berlin Spandau,243977
SK Berlin Steglitz-Zehlendorf,308697
SK Berlin Mitte,384172
SK Berlin Friedrichshain-Kreuzberg,289762
SK Berlin Tempelhof-Schöneberg,351644


In [12]:
cases_incidence = cases_sum.join(population)
cases_incidence[f"{period}-day-incidence-rate"] = (
    cases_incidence[f"{period}-day-sum"] / cases_incidence["population"] * 100_000
) # By convention the incedence rate is the total cases in the past x days / population * 100_000

In [13]:
cases_incidence.tail()

,14-day-sum,population,14-day-incidence-rate
Landkreis,,,
SK Worms,6,83330,7.200288
SK Wuppertal,86,354382,24.267598
SK Würzburg,3,127880,2.345949
SK Zweibrücken,1,34209,2.923207
StadtRegion Aachen,40,555465,7.201174


In [14]:
cases_incidence.loc['SK Hamburg']

14-day-sum               1.110000e+02
population               1.841179e+06
14-day-incidence-rate    6.028746e+00
Name: SK Hamburg, dtype: float64